In [1]:
import pandas as pd
from sqlalchemy import create_engine
import requests
disk_engine = create_engine('sqlite:///idmapping/idmapping.db')

In [21]:
# ORG_NAME = "human"
# ORG_NAME = "celegans"  # none
# ORG_NAME = "drosophila"
# ORG_NAME = "mouse"
# ORG_NAME = "rat"
ORG_NAME = "zebrafish"
file_name = f"out/{ORG_NAME}-no-confidence.txt"

In [10]:
def get_scope_id(pdb_id):

    api_url = f"https://data.rcsb.org/rest/v1/core/polymer_entity_instance/{pdb_id}/A"
    response = requests.get(api_url)
    test_js = response.json()
    for my_dict in test_js['rcsb_polymer_instance_feature']:
        if my_dict['type'] == 'SCOP':
            if 'feature_id' in my_dict:
                return(my_dict['feature_id'])
            elif 'annotation_id' in my_dict:
                return(my_dict['annotation_id'])
            else:
                return("")

# get_scope_id("1R21")

In [22]:
df = pd.read_csv(file_name, names=['domainId'])
df['domainId'] = df['domainId'].apply(lambda x: x.split("-")[1])
df.to_sql(f"no_conf_{ORG_NAME}", disk_engine)
print(len(df))

185


In [6]:
# disk_engine = create_engine('sqlite:///idmapping/idmapping-exp.db')
# file_name = "human-no-confidence-exp.txt"
# df = pd.read_csv(file_name, names=['domainId'])
# df['domainId'] = df['domainId'].apply(lambda x: x.split("-")[1])
# df.to_sql('no_conf_human_test-exp', disk_engine)

1

In [23]:
with disk_engine.connect() as con:

    rs_df = con.execute(
        f"""SELECT DISTINCT data.accessionId, data.'ID Type', data.'ID' 
        FROM no_conf_{ORG_NAME}
        LEFT JOIN data on data.accessionId = no_conf_{ORG_NAME}.domainId
        WHERE data.'ID Type' = 'PDB'
        """)
    rs_df

In [ ]:
rs_df['scope-id'] = rs_df['id'].apply(get_scope_id)
rs_df

In [ ]:
# pdb_df = pd.read_csv(f'out/{ORG_NAME}-mistake-pdb.csv', names=['uniprot', 'id-type', 'id'])
# pdb_df = pdb_df[['uniprot', 'id']]
# pdb_df['scope-id'] = pdb_df['id'].apply(get_scope_id)
# pdb_df